In [2]:
import pandas as pd
import shapely
import geopandas as gp
import json

In [3]:
layers = {}
layers["Borough"] = gp.GeoDataFrame.from_file("Borough Boundaries.geojson")
layers["Community"] = gp.GeoDataFrame.from_file("Community Districts.geojson")
layers["NTA"] = gp.GeoDataFrame.from_file("Neighborhood Tabulation Areas.geojson")
layers["Tract"] = gp.GeoDataFrame.from_file("2010 Census Tracts.geojson")
list = layers.keys()

In [4]:
for cov in list:
    envel = layers[cov].geometry.envelope
    enveljs = json.loads(envel.to_json())
    # An envelope field cannot be converted directly to
    # json, so create fields for the corners of the
    # envelope and add those
    # This will allow measuring the extent of what is displayed
    # and testing if a feature has any overlap with it.
    bounds = [[enveljs['features'][i]['bbox'][j] for i in range(len(enveljs['features']))] for j in range(4)]
    layers[cov]['minx'] = bounds[0]
    layers[cov]['miny'] = bounds[1]
    layers[cov]['maxx'] = bounds[2]
    layers[cov]['maxy'] = bounds[3]

In [5]:
dogs = pd.read_csv("NYC_Dog_Licensing_Dataset.csv", sep=';')
# Go through every combination of community district and tract and assign a correct tract id
small = dogs[['CommunityDistrict','boro_tract1','boro_tract2']].drop_duplicates().to_dict()

In [7]:
def overlap(b1, b2):
    # Test whether b2's envelope has any point inside b1's envelope
    if (((b2[0] > b1[0] and b2[0] < b1[2]) or (b2[2] > b1[0] and b2[2] < b1[2])) and 
        ((b2[1] > b1[1] and b2[1] < b1[3]) or (b2[3] > b1[1] and b2[3] < b1[3]))):
        return 1
    else:
        return 0

def compare_geos(comm, tr1, tr2):
    # If both tr1 and tr2 exist, check if one and not the other has any overlap
    # with comm
    box = layers["Community"][layers["Community"]["boro_cd"] == str(comm)].geometry.envelope
    cbox = json.loads(box.to_json())
    over = []
    for tr in tr1, tr2:
        box = layers["Tract"][layers["Tract"]["boro_ct2010"] == str(tr)].geometry.envelope
        tbox = json.loads(box.to_json())
        if overlap(cbox['bbox'], tbox['bbox']):
            over.append(1)
        else:
            over.append(0)
    return over
            
small['boro_ct2010'] = {}
for i in small['boro_tract1'].keys():
    # Start by checking if one value for tract exists but not the other
    chk1 = layers["Tract"][layers["Tract"]['boro_ct2010'] == str(small['boro_tract1'][i])]
    chk2 = layers["Tract"][layers["Tract"]['boro_ct2010'] == str(small['boro_tract2'][i])]
    if chk1.shape[0] == 1 and chk2.shape[0] == 0:
        small['boro_ct2010'][i] = str(small['boro_tract1'][i])
    elif chk1.shape[0] == 0 and chk2.shape[0] == 1:
        small['boro_ct2010'][i] = str(small['boro_tract2'][i])
    elif chk1.shape[0] == 1 and chk2.shape[0] == 1:
        # if both exist, check by geography
        if small['boro_tract1'][i] == small['boro_tract2'][i]:
            small['boro_ct2010'][i] = str(small['boro_tract1'][i])
        else:
            trInComm = compare_geos(small['CommunityDistrict'][i],
                                    small['boro_tract1'][i], small['boro_tract2'][i])
            if trInComm[0] == 1 and trInComm[1] == 0:
                small['boro_ct2010'][i] = str(small['boro_tract1'][i])
            elif trInComm[0] == 0 and trInComm[1] == 1:
                small['boro_ct2010'][i] = str(small['boro_tract2'][i])
            else:
                small['boro_ct2010'][i] = '0000000'
    else:
        # didn't happen :)
        small['boro_ct2010'][i] = '0000000'

In [8]:
# implement the process
fix_tracts = pd.DataFrame.from_dict(small)
fix_tracts.boro_ct2010 = fix_tracts.boro_ct2010.astype(str)
fix_tracts.boro_tract1 = fix_tracts.boro_tract1.astype(str)
fix_tracts.boro_tract2 = fix_tracts.boro_tract2.astype(str)
fix_tracts[(fix_tracts.boro_ct2010 == fix_tracts.boro_tract2) &
           (fix_tracts.boro_ct2010 != fix_tracts.boro_tract1)].shape

(46, 4)

In [9]:
dogs.CommunityDistrict = dogs.CommunityDistrict.astype(str)
dogs.boro_tract1 = dogs.boro_tract1.astype(str)
dogs.boro_tract2 = dogs.boro_tract2.astype(str)
# Most records are of the variety 501 === 050100 rather than 501 === 000501
# so first set all of them that way, then go through the exceptions
dogs['boro_ct2010'] = dogs.boro_tract1
# get the list of where the tract is of variety 501 === 000501
use_tract2 = fix_tracts[(fix_tracts.boro_ct2010 == fix_tracts.boro_tract2) &
                        (fix_tracts.boro_ct2010 != fix_tracts.boro_tract1)]
# iterate through that list and set the tract field for those community/tract combos
for ind, row in use_tract2.iterrows():
    dogs.loc[(dogs.CommunityDistrict == row['CommunityDistrict']) &
             (dogs.boro_tract2 == row['boro_tract2']), 'boro_ct2010'] = dogs.boro_tract2

In [10]:
boroughs = {"1" : "Manhattan", "2" : "Bronx", "3" : "Brooklyn",
            "4" : "Queens", "5": "Staten Island"}
# Set the borough name to a consistent value within each borough
for i in '1', '2', '3', '4', '5':
    dogs.loc[dogs.boro_ct2010.str[0] == i, 'Borough'] = boroughs[i]

In [11]:
# Aggregate the number of dogs within each geographic unit
aggdogs = {}
aggdogs['Borough'] = dogs.groupby('Borough').size()
aggdogs['Community'] = dogs.groupby('CommunityDistrict').size()
aggdogs['NTA'] = dogs.groupby('NTA').size()
aggdogs['Tract'] = dogs.groupby('boro_ct2010').size()
for k in list:
    aggdogs[k] = aggdogs[k].to_frame().rename(index=str, columns={0: 'numdogs'})

In [12]:
layer_cols = {'Borough': 'boro_name', 'Community': 'boro_cd',
              'NTA': 'ntacode', 'Tract': 'boro_ct2010'}
# Merge the aggregated number into the geocoverage for each geography
# and then export as a geojson
for cov in list:
    layers[cov][layer_cols[cov]] = layers[cov][layer_cols[cov]].astype(str)
    mrg = pd.merge(layers[cov], aggdogs[cov], how='left', left_on=layer_cols[cov], right_index=True)
    mrg['numdogs'].fillna(0, inplace=True)
    mrg.numdogs = mrg.numdogs.astype(int)
    js = mrg.to_json()
    fh = open('data/{}.geojson'.format(cov), 'w')
    fh.write(js)
